In [86]:
# importing necessary libraries
import pandas as pd
import yfinance as yf
from datetime import timedelta


In [87]:
spy_stocklist = pd.read_csv(r'raw_data/sp_500_historical_components.csv')

# Data cleaning
In this portion of the code we will 
1) Get the list of stocks that ever existed from sp_500_historical_components.csv from https://github.com/hanshof/sp500_constituents/blob/main/sp_500_historical_components.csv
2) Use yfinance library to get historical data (2012-2020) OHLCV, PE ratio, PB ratio from the lists of stocks, indicate their presence in the stock data at any point of time on a daily level. we get 2012 data for a buffer to calculate moving averages later on
3) Filter out stocks that newly entered the stock market before 2016, this is to ensure that we have sufficient training data for each stock

In [88]:

# Convert 'date' column to datetime
spy_stocklist['date'] = pd.to_datetime(spy_stocklist['date'])

# Ensure 'tickers' is string
spy_stocklist['tickers'] = spy_stocklist['tickers'].astype(str)
spy_stocklist

,date,tickers
0,1996-01-02,"AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD..."
1,1996-01-03,"AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD..."
2,1996-01-04,"AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD..."
3,1996-01-10,"AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD..."
4,1996-01-11,"AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD..."
...,...,...
3477,2025-08-19,"A,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM,ADP..."
3478,2025-08-20,"A,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM,ADP..."
3479,2025-08-21,"A,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM,ADP..."
3480,2025-08-22,"A,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM,ADP..."


In [89]:
# Filter date range for 2012-01-01 to 2021-01-07, we include the year 2012 to introduce a buffer for moving averages calculation
spy_stocklist_filtered = spy_stocklist[(spy_stocklist['date'] >= '2013-01-01') & (spy_stocklist['date'] <= '2020-12-31')]

In [90]:
spy_stocklist_filtered_expanded = (
    spy_stocklist_filtered.assign(ticker=spy_stocklist_filtered["tickers"].str.split(","))
      .explode("ticker")
      .drop(columns="tickers")
      .reset_index(drop=True)
)

spy_stocklist_filtered_expanded

,date,ticker
0,2013-01-02,A
1,2013-01-02,AAPL
2,2013-01-02,ABBV
3,2013-01-02,ABC
4,2013-01-02,ABT
...,...,...
343971,2020-12-21,YUM
343972,2020-12-21,ZBH
343973,2020-12-21,ZBRA
343974,2020-12-21,ZION


In [91]:
# there are certain cases where a stock may have rebranded, we will update the list accordingly later
## TODO
updated_ticker_mapping = {
    'GOOGL': 'GOOG',  
    'FB': 'META',     
    'TWTR': 'X',      
}

In [92]:
## defining a function to replace the tickers

def update_tickers_in_df_from_dict(df, column, ticker_map):

    df_updated = df.copy()

    # Replace using pandas built-in mapping
    df_updated[column] = df_updated[column].replace(ticker_map)

    return df_updated

In [93]:
spy_stocklist_final = update_tickers_in_df_from_dict(spy_stocklist_filtered_expanded, "ticker", updated_ticker_mapping)
spy_stocklist_final[spy_stocklist_final['ticker'].isin(updated_ticker_mapping.values())]

,date,ticker
185,2013-01-02,GOOG
448,2013-01-02,X
643,2013-01-03,GOOG
906,2013-01-03,X
1101,2013-01-08,GOOG
...,...,...
343416,2020-11-17,X
343649,2020-12-21,META
343674,2020-12-21,GOOG
343675,2020-12-21,GOOG


In [94]:
### getting list of unique tickers that ever entered the SP500 between 2013-2020
unique_tickers = spy_stocklist_final['ticker'].unique().tolist()
print(f"Total unique tickers in S&P 500 from 2013 to 2020: {len(unique_tickers)}")
unique_tickers

Total unique tickers in S&P 500 from 2013 to 2020: 639


['A',
 'AAPL',
 'ABBV',
 'ABC',
 'ABT',
 'ACN',
 'ADBE',
 'ADI',
 'ADM',
 'ADP',
 'ADSK',
 'AEE',
 'AEP',
 'AES',
 'AET',
 'AFL',
 'AIG',
 'AIV',
 'AIZ',
 'AKAM',
 'ALL',
 'ALXN',
 'AMAT',
 'AMD',
 'AMGN',
 'AMP',
 'AMT',
 'AMZN',
 'AN',
 'ANDV',
 'ANF',
 'ANTM',
 'AON',
 'APA',
 'APC',
 'APD',
 'APH',
 'APOL',
 'APTV',
 'ARG',
 'ATI',
 'AVB',
 'AVP',
 'AVY',
 'AXP',
 'AZO',
 'BA',
 'BAC',
 'BAX',
 'BBBY',
 'BBT',
 'BBY',
 'BCR',
 'BDX',
 'BEN',
 'BF-B',
 'BIG',
 'BIIB',
 'BK',
 'BKNG',
 'BLK',
 'BLL',
 'BMS',
 'BMY',
 'BRCM',
 'BRK-B',
 'BSX',
 'BTUUQ',
 'BWA',
 'BXP',
 'C',
 'CA',
 'CAG',
 'CAH',
 'CAM',
 'CAT',
 'CB',
 'CBRE',
 'CBS',
 'CCI',
 'CCL',
 'CELG',
 'CERN',
 'CF',
 'CFN',
 'CHRW',
 'CI',
 'CINF',
 'CL',
 'CLF',
 'CLX',
 'CMA',
 'CMCSA',
 'CME',
 'CMG',
 'CMI',
 'CMS',
 'CNP',
 'CNX',
 'COF',
 'COG',
 'COL',
 'COP',
 'COST',
 'COV',
 'CPB',
 'CRM',
 'CSCO',
 'CSX',
 'CTAS',
 'CTL',
 'CTSH',
 'CTXS',
 'CVC',
 'CVS',
 'CVX',
 'D',
 'DD',
 'DE',
 'DF',
 'DFS',
 'DG',
 'DGX',


In [95]:
### now we will use the yfinance library to get the historical data for these tickers that ever existed in the SP500 between 2013-2020
def get_stock_data(ticker, start_date, end_date):
    try:
        # Convert string to datetime so we can safely add 1 day, because yfinance end date is exclusive we add one day to include the end date
        end_date_dt = pd.to_datetime(end_date) + pd.Timedelta(days=1)
        
        stock = yf.Ticker(ticker)
        stock_data = stock.history(
            start=start_date,
            end=end_date_dt,
            auto_adjust=True,  # adjust for stock splits/dividends
            actions=True       # include Dividends and Stock Splits columns
        )
        
        if stock_data.empty:
            return pd.DataFrame()
        
        stock_data["ticker"] = ticker
        return stock_data.reset_index()

    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return pd.DataFrame()

In [96]:
start_date = '2012-01-01' # we start from 2012 to have a buffer for moving averages calculation
end_date = '2020-12-31' # we end at 2020-12-31, 2019-2020 will be held out for stress testing
yfinance_data = []
failed_tickers = []
successful_tickers = []
for ticker in unique_tickers:
    print(f"Fetching data for {ticker}...")
    ticker_data = get_stock_data(ticker, start_date, end_date)
    if not ticker_data.empty:
        yfinance_data.append(ticker_data)
        successful_tickers.append(ticker)
    else:
        failed_tickers.append(ticker)

    

Fetching data for A...
Fetching data for AAPL...
Fetching data for ABBV...
Fetching data for ABC...


$ABC: possibly delisted; no timezone found


Fetching data for ABT...
Fetching data for ACN...
Fetching data for ADBE...
Fetching data for ADI...
Fetching data for ADM...
Fetching data for ADP...
Fetching data for ADSK...
Fetching data for AEE...
Fetching data for AEP...
Fetching data for AES...
Fetching data for AET...
Fetching data for AFL...
Fetching data for AIG...
Fetching data for AIV...
Fetching data for AIZ...
Fetching data for AKAM...
Fetching data for ALL...
Fetching data for ALXN...


$ALXN: possibly delisted; no timezone found


Fetching data for AMAT...
Fetching data for AMD...
Fetching data for AMGN...
Fetching data for AMP...
Fetching data for AMT...
Fetching data for AMZN...
Fetching data for AN...
Fetching data for ANDV...
Fetching data for ANF...
Fetching data for ANTM...


...

Fetching data for NOW...
Fetching data for VIAC...


$VIAC: possibly delisted; no timezone found


Fetching data for WRB...
Fetching data for ODFL...
Fetching data for TFC...
Fetching data for J...
Fetching data for LYV...
Fetching data for STE...
Fetching data for ZBRA...
Fetching data for PAYC...
Fetching data for TT...
Fetching data for CARR...
Fetching data for OTIS...
Fetching data for RTX...
Fetching data for HWM...
Fetching data for DPZ...
Fetching data for DXCM...
Fetching data for WST...
Fetching data for BIO...
Fetching data for TDY...
Fetching data for TYL...
Fetching data for LUMN...
Fetching data for CTLT...


$CTLT: possibly delisted; no timezone found


Fetching data for ETSY...
Fetching data for POOL...
Fetching data for VNT...
Fetching data for VTRS...
Fetching data for TSLA...


In [97]:
# combine all individual DataFrames into one big DataFrame
yfinance_df = pd.concat(yfinance_data, ignore_index=True)

# optional: ensure 'Date' is a proper datetime column
yfinance_df["Date"] = pd.to_datetime(yfinance_df["Date"])

# preview result
yfinance_df.drop(columns=['Capital Gains'], inplace=True)
yfinance_df.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,ticker
0,2012-01-03 00:00:00-05:00,22.776927,23.507771,22.713375,23.183659,4156394,0.0,0.0,A
1,2012-01-04 00:00:00-05:00,22.973940,23.107398,22.618049,22.999359,4651845,0.0,0.0,A
2,2012-01-05 00:00:00-05:00,22.802352,23.717496,22.700669,23.514132,6842651,0.0,0.0,A
3,2012-01-06 00:00:00-05:00,23.571317,23.870010,23.393373,23.768328,4711400,0.0,0.0,A
4,2012-01-09 00:00:00-05:00,23.908147,24.416561,23.812819,24.391140,4429563,0.0,0.0,A


In [98]:
# checking for stock split price adjustment, for example apple, check the dates around 2020-08-31 when apple had a 4-for-1 stock split
yfinance_df[yfinance_df['Date'].between('2020-08-24', '2020-09-07') & (yfinance_df['ticker'] == 'AAPL')]

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,ticker
4439,2020-08-24 00:00:00-04:00,125.186351,125.271473,120.556219,122.423836,345937600,0.0,0.0,AAPL
4440,2020-08-25 00:00:00-04:00,121.295478,121.764813,119.695353,121.419495,211495600,0.0,0.0,AAPL
4441,2020-08-26 00:00:00-04:00,122.737524,123.527857,121.669963,123.070679,163022400,0.0,0.0,AAPL
4442,2020-08-27 00:00:00-04:00,123.673777,124.006932,120.454076,121.599457,155552400,0.0,0.0,AAPL
4443,2020-08-28 00:00:00-04:00,122.574593,122.992862,121.178745,121.402473,187630000,0.0,0.0,AAPL
4444,2020-08-31 00:00:00-04:00,124.099333,127.426026,122.562438,125.519493,225702700,0.0,4.0,AAPL
4445,2020-09-01 00:00:00-04:00,129.137984,131.122336,126.968828,130.519241,151948100,0.0,0.0,AAPL
4446,2020-09-02 00:00:00-04:00,133.836227,134.215586,123.535150,127.815102,200119000,0.0,0.0,AAPL
4447,2020-09-03 00:00:00-04:00,123.447633,125.324972,117.212508,117.582138,257599600,0.0,0.0,AAPL
4448,2020-09-04 00:00:00-04:00,116.794224,120.325187,107.864675,117.659943,332607200,0.0,0.0,AAPL


In [99]:
# Ensure both columns are in the same format
yfinance_df["Date"] = pd.to_datetime(yfinance_df["Date"]).dt.date
spy_stocklist_filtered_expanded["date"] = pd.to_datetime(spy_stocklist_filtered_expanded["date"]).dt.date

# Create a set of (date, ticker) pairs for fast lookup
spy_set = set(zip(spy_stocklist_filtered_expanded["date"], spy_stocklist_filtered_expanded["ticker"]))

# Add indicator column: 1 if (Date, Ticker) is in SPY list, else 0
yfinance_df["is_in_sp500"] = [
    1 if (d, t) in spy_set else 0 for d, t in zip(yfinance_df["Date"], yfinance_df["ticker"])
]

yfinance_df.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,ticker,is_in_sp500
0,2012-01-03,22.776927,23.507771,22.713375,23.183659,4156394,0.0,0.0,A,0
1,2012-01-04,22.973940,23.107398,22.618049,22.999359,4651845,0.0,0.0,A,0
2,2012-01-05,22.802352,23.717496,22.700669,23.514132,6842651,0.0,0.0,A,0
3,2012-01-06,23.571317,23.870010,23.393373,23.768328,4711400,0.0,0.0,A,0
4,2012-01-09,23.908147,24.416561,23.812819,24.391140,4429563,0.0,0.0,A,0


In [100]:
# filtering stocks that entered the stock market after 2016-01-01 to ensure we have enough training data per stock

first_trade = yfinance_df.groupby("ticker")["Date"].min().reset_index()
first_trade["Date"] = pd.to_datetime(first_trade["Date"])
first_trade.columns = ["ticker", "first_trade_date"]

# keep only tickers that started before or on 2015-12-31
eligible_tickers = first_trade[first_trade["first_trade_date"] <= "2015-12-31"]["ticker"]

yfinance_df_filtered_relavant_stocks = filtered_df = yfinance_df[yfinance_df["ticker"].isin(eligible_tickers)].copy()


print(f"Total tickers after filtering: {eligible_tickers.nunique()}, tickers filtered out: {len(unique_tickers) - eligible_tickers.nunique()}")

Total tickers after filtering: 506, tickers filtered out: 133


In [101]:
# getting daily returns, monthly returns and adding to the dataframe
# we take 21 trading days as approximately 1 month

yfinance_df_filtered_relavant_stocks.sort_values(by=['ticker', 'Date'], inplace=True)
yfinance_df_filtered_relavant_stocks['daily_return'] = yfinance_df_filtered_relavant_stocks.groupby('ticker')['Close'].pct_change()
yfinance_df_filtered_relavant_stocks['monthly_return'] = yfinance_df_filtered_relavant_stocks.groupby('ticker')['Close'].pct_change(periods=21)

In [102]:
# getting monthly standard deviation (volatility) (TODO)
yfinance_df_filtered_relavant_stocks["monthly_var_3"] = (
    yfinance_df_filtered_relavant_stocks.groupby("ticker")["monthly_return"]
      .transform(lambda x: x.rolling(window=3, min_periods=3).var(ddof=1))
)

yfinance_df_filtered_relavant_stocks["daily_vol_20"] = (
    yfinance_df_filtered_relavant_stocks
    .groupby("ticker")["daily_return"]
    .transform(lambda x: x.rolling(window=20, min_periods=20).std(ddof=1))
)

### EDA for new stocks that entered after our 2013 cut off
- the problem with these stocks is that they might not have enough historical data to compute fields such as:
1) monthly returns
2) monthly variance (3 months)
3) EMA/SMA

In [103]:
start_date = pd.Timestamp("2013-01-01")

# ensure Date is Timestamp
yfinance_df_filtered_relavant_stocks["Date"] = pd.to_datetime(yfinance_df_filtered_relavant_stocks["Date"])

# find earliest date per ticker
first_dates = (
    yfinance_df_filtered_relavant_stocks
    .groupby("ticker")["Date"]
    .min()
    .reset_index(name="first_date")
)

# flag tickers that started trading after 2013-01-01
first_dates["entered_after_2013"] = (first_dates["first_date"] > start_date).astype(int)

# count and preview
num_after = first_dates["entered_after_2013"].sum()
num_total = len(first_dates)

print(f"{num_after} out of {num_total} tickers started trading after {start_date.date()} before 2015-12-31.")
print(first_dates.query("entered_after_2013 == 1").head())

25 out of 506 tickers started trading after 2013-01-01 before 2015-12-31.
   ticker first_date  entered_after_2013
4    ABBV 2013-01-02                   1
26   ALLE 2013-11-18                   1
38   ANET 2014-06-06                   1
88    CDW 2013-06-27                   1
91    CFG 2014-09-24                   1


### Creating new features
#### Moving averages
https://www.investopedia.com/ask/answers/122414/what-are-most-common-periods-used-creating-moving-average-ma-lines.asp we will use short:20 days, medium: 50 days, long: 100 days moving averages
1) Simple moving average SMA: SMA_20, SMA_50, SMA_100
2) Exponential Moving Average EMA https://www.investopedia.com/terms/e/ema.asp#toc-formula-for-exponential-moving-average-ema: EMA_20, EMA_50, EMA_100

#### RSI and MACD
1) Relative Strength Index 14 days https://www.investopedia.com/terms/r/rsi.asp : RSI
2) Moving Average Convergence/Divergence indicator: https://www.investopedia.com/terms/m/macd.asp: MACD_26, MACD_12, MACD_9 

#### Stock Splits
1) Forward stock split (Stock split > 1): Commonly known as a bullish indicator which provides more liquidity (when fractional shares werent common)
2) Reverse stock split (0 < Stock split < 1): Bearish indicator

In [104]:
def sma(series: pd.Series, window: int) -> pd.Series:
    """Simple Moving Average."""
    return series.rolling(window, min_periods=window).mean()

In [105]:
def ema(series: pd.Series, span: int) -> pd.Series:
    """Exponential Moving Average."""
    return series.ewm(span=span, adjust=False, min_periods=span).mean()

In [106]:
def rsi_wilder(series: pd.Series, period: int = 14) -> pd.Series:
    """Wilder's RSI (default 14)."""
    delta = series.diff()
    gain  = delta.clip(lower=0)
    loss  = -delta.clip(upper=0)
    avg_gain = gain.ewm(alpha=1/period, adjust=False, min_periods=period).mean()
    avg_loss = loss.ewm(alpha=1/period, adjust=False, min_periods=period).mean()
    rs = avg_gain / avg_loss
    return 100 - (100 / (1 + rs))

In [107]:
def macd(series: pd.Series, fast: int = 12, slow: int = 26, signal: int = 9):
    """
    MACD parts: returns DataFrame with MACD_Line, MACD_Signal, MACD_Hist.
    """
    ema_fast = ema(series, fast)
    ema_slow = ema(series, slow)
    macd_line = ema_fast - ema_slow
    macd_signal = macd_line.ewm(span=signal, adjust=False, min_periods=signal).mean()
    macd_hist = macd_line - macd_signal
    return pd.DataFrame(
        {"MACD_Line": macd_line, "MACD_Signal": macd_signal, "MACD_Hist": macd_hist},
        index=series.index
    )

In [108]:
temp_df = yfinance_df_filtered_relavant_stocks.copy()

# Ensure types/order; compute per-ticker
temp_df['Date'] = pd.to_datetime(temp_df['Date'])
temp_df = temp_df.drop_duplicates(subset=['ticker','Date']).sort_values(['ticker','Date'])


g = temp_df.groupby('ticker', group_keys=False)

# SMA 20/50/100
for w in [20, 50, 100]:
    temp_df[f'SMA_{w}'] = g['Close'].transform(lambda s, w=w: sma(s, w))

# EMA 20/50/100
for w in [20, 50, 100]:
    temp_df[f'EMA_{w}'] = g['Close'].transform(lambda s, w=w: ema(s, w))

# RSI 14
temp_df['RSI'] = g['Close'].transform(rsi_wilder)

# MACD (12,26,9)
macd_df = g['Close'].apply(macd)
temp_df = temp_df.join(macd_df)

# Stock Splits
temp_df["is_forward_split"] = (yfinance_df["Stock Splits"] > 1).astype(int)
temp_df["is_reverse_split"] = ((yfinance_df["Stock Splits"] > 0) & 
                                   (yfinance_df["Stock Splits"] < 1)).astype(int) 

SP500_all_stock_data = temp_df


SP500_all_stock_data

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,ticker,is_in_sp500,...,SMA_100,EMA_20,EMA_50,EMA_100,RSI,MACD_Line,MACD_Signal,MACD_Hist,is_forward_split,is_reverse_split
0,2012-01-03,22.776927,23.507771,22.713375,23.183659,4156394,0.0,0.0,A,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
1,2012-01-04,22.973940,23.107398,22.618049,22.999359,4651845,0.0,0.0,A,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
2,2012-01-05,22.802352,23.717496,22.700669,23.514132,6842651,0.0,0.0,A,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
3,2012-01-06,23.571317,23.870010,23.393373,23.768328,4711400,0.0,0.0,A,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
4,2012-01-09,23.908147,24.416561,23.812819,24.391140,4429563,0.0,0.0,A,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
839121,2020-12-24,153.424234,155.295024,153.357085,154.191742,417400,0.0,0.0,ZTS,0,...,154.709181,154.477097,154.663482,151.699308,48.711727,-0.298372,-0.514775,0.216403,0,0
839122,2020-12-28,154.882506,156.216045,153.798400,155.793915,1522400,0.0,0.0,ZTS,0,...,154.755875,154.602508,154.707812,151.780389,52.813311,-0.160577,-0.443936,0.283359,0,0
839123,2020-12-29,156.580649,158.393879,155.803537,156.494293,1188400,0.0,0.0,ZTS,0,...,154.774983,154.782678,154.777871,151.873734,54.525340,0.005082,-0.354132,0.359214,0,0
839124,2020-12-30,156.868410,158.106020,156.532636,157.597549,1009000,0.0,0.0,ZTS,0,...,154.828599,155.050761,154.888446,151.987077,57.161972,0.222823,-0.238741,0.461565,0,0


In [109]:
# we dont want to backward fill the data as it may introduce lookahead bias, 
# nor do we want to delete the entries, so for stocks that newly entered the SP500 with insufficient data we fill with 0
SP500_all_stock_data.fillna(0, inplace=True)
SP500_all_stock_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,ticker,is_in_sp500,...,SMA_100,EMA_20,EMA_50,EMA_100,RSI,MACD_Line,MACD_Signal,MACD_Hist,is_forward_split,is_reverse_split
0,2012-01-03,22.776927,23.507771,22.713375,23.183659,4156394,0.0,0.0,A,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
1,2012-01-04,22.973940,23.107398,22.618049,22.999359,4651845,0.0,0.0,A,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
2,2012-01-05,22.802352,23.717496,22.700669,23.514132,6842651,0.0,0.0,A,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
3,2012-01-06,23.571317,23.870010,23.393373,23.768328,4711400,0.0,0.0,A,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
4,2012-01-09,23.908147,24.416561,23.812819,24.391140,4429563,0.0,0.0,A,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0


In [110]:
# for daily prices we use spreads instead of absolute prices
SP500_all_stock_data['high_low_spread'] = (SP500_all_stock_data['High'] - SP500_all_stock_data['Low'])/SP500_all_stock_data['Close']
SP500_all_stock_data['open_close_spread'] = (SP500_all_stock_data['Open'] - SP500_all_stock_data['Close'])/SP500_all_stock_data['Close']

# for moving averages we use moving averages of returns instead of absolute prices
for w in [20, 50, 100]:
    SP500_all_stock_data[f"SMA_ret_{w}"] = g["daily_return"].transform(lambda s, w=w: s.rolling(window=w, min_periods=w).mean())

# EMA (Exponential Moving Average) of daily returns
for w in [20, 50, 100]:
    SP500_all_stock_data[f"EMA_ret_{w}"] = g["daily_return"].transform(lambda s, w=w: s.ewm(span=w, adjust=False, min_periods=w).mean())

In [111]:
# filter for stocks from 2013-01-01 onwards 
SP500_all_stock_data_final = SP500_all_stock_data[SP500_all_stock_data['Date'] >= pd.to_datetime('2013-01-01')]
SP500_all_stock_data_final

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,ticker,is_in_sp500,...,is_forward_split,is_reverse_split,high_low_spread,open_close_spread,SMA_ret_20,SMA_ret_50,SMA_ret_100,EMA_ret_20,EMA_ret_50,EMA_ret_100
250,2013-01-02,27.067739,27.067739,26.413031,26.881598,8790205,0.0,0.0,A,1,...,0,0,0.024355,0.006924,0.005231,0.002119,0.000592,0.004949,0.003209,0.001800
251,2013-01-03,26.920105,27.048481,26.689031,26.977875,5751791,0.0,0.0,A,1,...,0,0,0.013324,-0.002141,0.005134,0.002936,0.000526,0.004818,0.003224,0.001835
252,2013-01-04,27.048494,27.568411,26.868769,27.510641,6432897,0.0,0.0,A,0,...,0,0,0.025432,-0.016799,0.005426,0.003817,0.000689,0.006240,0.003872,0.002190
253,2013-01-07,27.343746,27.472121,27.202534,27.311653,3589505,0.0,0.0,A,0,...,0,0,0.009871,0.001175,0.005504,0.003533,0.000683,0.004957,0.003436,0.002003
254,2013-01-08,27.260303,27.459284,27.022810,27.093416,3896925,0.0,0.0,A,1,...,0,0,0.016110,0.006160,0.005770,0.003645,0.000640,0.003724,0.002988,0.001805
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
839121,2020-12-24,153.424234,155.295024,153.357085,154.191742,417400,0.0,0.0,ZTS,0,...,0,0,0.012568,-0.004978,0.000207,-0.000220,0.000439,-0.000127,-0.000022,0.000498
839122,2020-12-28,154.882506,156.216045,153.798400,155.793915,1522400,0.0,0.0,ZTS,0,...,0,0,0.015518,-0.005850,0.000327,0.000070,0.000409,0.000875,0.000387,0.000694
839123,2020-12-29,156.580649,158.393879,155.803537,156.494293,1188400,0.0,0.0,ZTS,0,...,0,0,0.016552,0.000552,0.000889,0.000091,0.000225,0.001220,0.000548,0.000770
839124,2020-12-30,156.868410,158.106020,156.532636,157.597549,1009000,0.0,0.0,ZTS,0,...,0,0,0.009984,-0.004627,0.000936,0.000667,0.000447,0.001775,0.000803,0.000894


#### Stationarity checks
we will use ADF test for stationarity checks and eliminate serial correlation through transformations if required

In [112]:
from statsmodels.tsa.stattools import adfuller
import numpy as np

In [113]:
def check_adf_all_columns(df):
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    results = []

    for col in numeric_cols:
        series = df[col].dropna()
        try:
            p_val = adfuller(series, autolag="AIC")[1]
            verdict = "stationary" if p_val < 0.05 else "non_stationary"
        except Exception as e:
            p_val = np.nan
            verdict = f"error: {e}"

        results.append({
            "column": col,
            "ADF_p": p_val,
            "Verdict": verdict
        })

    return pd.DataFrame(results)

In [114]:
# Using only one stock (appl) for ADF because my kernel keeps crashing and its computationally expensive :"
appl_stock_data = SP500_all_stock_data_final[SP500_all_stock_data_final['ticker'] == 'AAPL']
adf_results = check_adf_all_columns(appl_stock_data)
display(adf_results)

,column,ADF_p,Verdict
0,Open,1.000000e+00,non_stationary
1,High,1.000000e+00,non_stationary
2,Low,1.000000e+00,non_stationary
3,Close,1.000000e+00,non_stationary
4,Volume,3.941899e-04,stationary
5,Stock Splits,0.000000e+00,stationary
6,is_in_sp500,1.621880e-03,stationary
7,daily_return,8.005636e-27,stationary
8,monthly_return,4.571905e-09,stationary
9,monthly_var_3,9.980689e-05,stationary


In [115]:
# dropping non-stationary columns
SP500_all_stock_data_final.drop(columns=['Open', 'High', 'Low', 'Close', 'SMA_20', 'SMA_50', 'SMA_100', 'EMA_20', 'EMA_50', 'EMA_100'], inplace=True)
SP500_all_stock_data_final

/var/folders/sr/m5w47sf11d525m54586gg3c00000gn/T/ipykernel_20860/1777297170.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SP500_all_stock_data_final.drop(columns=['Open', 'High', 'Low', 'Close', 'SMA_20', 'SMA_50', 'SMA_100', 'EMA_20', 'EMA_50', 'EMA_100'], inplace=True)


,Date,Volume,Dividends,Stock Splits,ticker,is_in_sp500,daily_return,monthly_return,monthly_var_3,daily_vol_20,...,is_forward_split,is_reverse_split,high_low_spread,open_close_spread,SMA_ret_20,SMA_ret_50,SMA_ret_100,EMA_ret_20,EMA_ret_50,EMA_ret_100
250,2013-01-02,8790205,0.0,0.0,A,1,0.022960,0.096439,0.000508,0.016544,...,0,0,0.024355,0.006924,0.005231,0.002119,0.000592,0.004949,0.003209,0.001800
251,2013-01-03,5751791,0.0,0.0,A,1,0.003582,0.111103,0.000424,0.016548,...,0,0,0.013324,-0.002141,0.005134,0.002936,0.000526,0.004818,0.003224,0.001835
252,2013-01-04,6432897,0.0,0.0,A,0,0.019748,0.126805,0.000231,0.016762,...,0,0,0.025432,-0.016799,0.005426,0.003817,0.000689,0.006240,0.003872,0.002190
253,2013-01-07,3589505,0.0,0.0,A,0,-0.007233,0.103319,0.000143,0.016695,...,0,0,0.009871,0.001175,0.005504,0.003533,0.000683,0.004957,0.003436,0.002003
254,2013-01-08,3896925,0.0,0.0,A,1,-0.007991,0.104213,0.000177,0.016420,...,0,0,0.016110,0.006160,0.005770,0.003645,0.000640,0.003724,0.002988,0.001805
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
839121,2020-12-24,417400,0.0,0.0,ZTS,0,0.005442,-0.002111,0.000204,0.009383,...,0,0,0.012568,-0.004978,0.000207,-0.000220,0.000439,-0.000127,-0.000022,0.000498
839122,2020-12-28,1522400,0.0,0.0,ZTS,0,0.010391,0.013733,0.000429,0.009502,...,0,0,0.015518,-0.005850,0.000327,0.000070,0.000409,0.000875,0.000387,0.000694
839123,2020-12-29,1188400,0.0,0.0,ZTS,0,0.004496,0.010219,0.000069,0.009394,...,0,0,0.016552,0.000552,0.000889,0.000091,0.000225,0.001220,0.000548,0.000770
839124,2020-12-30,1009000,0.0,0.0,ZTS,0,0.007050,0.024255,0.000053,0.009423,...,0,0,0.009984,-0.004627,0.000936,0.000667,0.000447,0.001775,0.000803,0.000894


In [116]:
SP500_all_stock_data_final.isna().sum()

Date                    0
Volume                  0
Dividends               0
Stock Splits            0
ticker                  0
is_in_sp500             0
daily_return            0
monthly_return          0
monthly_var_3           0
daily_vol_20            0
RSI                     0
MACD_Line               0
MACD_Signal             0
MACD_Hist               0
is_forward_split        0
is_reverse_split        0
high_low_spread         0
open_close_spread       0
SMA_ret_20            500
SMA_ret_50           1250
SMA_ret_100          2547
EMA_ret_20            500
EMA_ret_50           1250
EMA_ret_100          2547
dtype: int64

#### Covid hold out set 
https://en.wikipedia.org/wiki/2020_stock_market_crash#:~:text=Though%20the%20crash%20began%20on,13%25%20in%20most%20global%20markets.

Covid stock crash happened on: 20 Feb
we will use 2013-01-01 to 2020-02-19 data as our training test set and 2020-02-19 to 2020-12-12 as our holdout set

Purging will happen later using 100 days during time-series CV

In [117]:
# splitting the data appropriately into training, test and covid stress test data 
# Covid start https://en.wikipedia.org/wiki/2020_stock_market_crash#:~:text=Though%20the%20crash%20began%20on,13%25%20in%20most%20global%20markets.
COVID_start_date = '2020-02-20'
training_data =  SP500_all_stock_data_final[SP500_all_stock_data_final['Date'] < pd.to_datetime(COVID_start_date)]
covid_stress_test_data = SP500_all_stock_data_final[SP500_all_stock_data_final['Date'] >= pd.to_datetime(COVID_start_date)]